In [ ]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
df = pd.read_csv('./housing_data.csv', parse_dates=['sold_date'])
df = df.drop_duplicates()
df = df.dropna()
df.info()

In [ ]:
def fix_state(df):
    df1 = df.loc[df.state != 'VIC'].copy()
    df1['suburb'] = df1['suburb'] + ' ' + df['state']
    df1 = df1.drop('state', axis=1)
    df1.postcode = df1.postcode.str.strip()
    df1[['state', 'postcode']] = df1.postcode.str.split(" ", expand=True)
    return df1

df1 = fix_state(df)
df1.info()

In [ ]:
df_final = pd.concat([df , df1])
df_final = df_final.drop_duplicates(subset='address', keep='last')

In [ ]:
df_final['country'] = 'AUSTRALIA'

In [ ]:
df_final['whole_address'] = df_final[['address', 'suburb', 'state', 'postcode', 'country']].T.agg(', '.join)
df_final

In [ ]:
geolocator = Nominatim(user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.109 Safari/537.36')

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df_final['location'] = df_final['whole_address'].apply(geocode)
df_final

# df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
